In [33]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('D:\Study\深度学习\日月光华-tensorflow入门与实战资料\数据集\小型CSV数据/Tweets.csv')

In [3]:
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
data = data[['airline_sentiment', 'text']]

In [5]:
data.airline_sentiment.unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [6]:
data.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [7]:
data_p = data[data.airline_sentiment == 'positive']

In [8]:
data_n = data[data.airline_sentiment == 'negative']

In [9]:
data_n = data_n.iloc[:len(data_p)]

In [10]:
len(data_n), len(data_p)

(2363, 2363)

In [11]:
data = pd.concat([data_n, data_p])

In [12]:
data = data.sample(len(data)) #乱序

In [13]:
data['review'] = (data.airline_sentiment == 'positive').astype('int')

In [14]:
del data['airline_sentiment']

tf.keras.layers.Embedding  把文本向量化

In [15]:
import re

In [16]:
token = re.compile('[A-Za-z]+|[!?,.()]')

In [17]:
def reg_text(text):#大写变成小写
    new_text = token.findall(text)
    new_text = [word.lower() for word in new_text]
    return new_text

In [18]:
data['text'] = data.text.apply(reg_text)

In [19]:
word_set = set()
for text in data.text:
    for word in text:
        word_set.add(word) 

In [20]:
max_word = len(word_set) + 1
max_word

7101

In [21]:
word_list = list(word_set)

In [22]:
word_list.index('spending')

4796

In [23]:
word_index =  dict((word, word_list.index(word) + 1) for word in word_list)

In [24]:
word_index

{'inspiring': 1,
 'closepwcs': 2,
 'flightn': 3,
 'chief': 4,
 'completely': 5,
 'feet': 6,
 'bitch': 7,
 'lmfao': 8,
 'impatience': 9,
 'baby': 10,
 'scared': 11,
 'term': 12,
 'throwback': 13,
 'understanding': 14,
 'listened': 15,
 'miranda': 16,
 'weighed': 17,
 'olds': 18,
 'walked': 19,
 'internationally': 20,
 'stickingtodelta': 21,
 'sits': 22,
 'continually': 23,
 'vinyl': 24,
 'trips': 25,
 'ilookyoung': 26,
 'watch': 27,
 'coupons': 28,
 'juan': 29,
 'wiped': 30,
 'absolute': 31,
 'sales': 32,
 'vjfv': 33,
 'sympathetic': 34,
 'wreck': 35,
 'flyitforward': 36,
 'consolation': 37,
 'bwood': 38,
 'reissued': 39,
 'tmm': 40,
 'refueled': 41,
 'honor': 42,
 'feelbetter': 43,
 'stays': 44,
 'operator': 45,
 'hotspot': 46,
 'bish': 47,
 'pales': 48,
 'smoothest': 49,
 'left': 50,
 'improvement': 51,
 'worst': 52,
 'heartlanta': 53,
 'entering': 54,
 'sincere': 55,
 'speaks': 56,
 'carol': 57,
 'more': 58,
 'laptop': 59,
 'na': 60,
 'consideration': 61,
 'jammed': 62,
 'qf': 63,
 '

In [25]:
data_ok = data.text.apply(lambda x: [word_index.get(word, 0) for word in x])

In [26]:
len(data_ok.iloc[2])

14

In [27]:
maxlen = max(len(x) for x in data_ok)

In [28]:
maxlen

40

In [29]:
data_ok = keras.preprocessing.sequence.pad_sequences(data_ok.values, maxlen=maxlen)

In [34]:
data_ok.shape

(4726, 40)

In [39]:
model=keras.Sequential()
model.add(layers.Embedding(max_word, 50, input_length=maxlen))
model.add(layers.Conv1D(32,7,activation='relu',padding='same'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32,7,activation='relu',padding='same'))
model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(1))

In [40]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 50)            355050    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 40, 32)            11232     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 13, 32)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 13, 32)            7200      
_________________________________________________________________
global_average_pooling1d_1 ( (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 373,515
Trainable params: 373,515
Non-trainable params: 0
________________________________________________

In [41]:
model.compile(optimizer=keras.optimizers.RMSprop(),
             loss='binary_crossentropy',
             metrics=['acc'])

In [42]:
history=model.fit(data_ok,data.review.values,epochs=10,batch_size=128,validation_split=0.3)

Epoch 1/10
26/26 [==============================] - 1s 20ms/step - loss: 0.7245 - acc: 0.6439 - val_loss: 0.5547 - val_acc: 0.7278
Epoch 2/10
26/26 [==============================] - 0s 8ms/step - loss: 0.4367 - acc: 0.8304 - val_loss: 0.2640 - val_acc: 0.9020
Epoch 3/10
26/26 [==============================] - 0s 9ms/step - loss: 0.2218 - acc: 0.9278 - val_loss: 0.2600 - val_acc: 0.9097
Epoch 4/10
26/26 [==============================] - 0s 9ms/step - loss: 0.1585 - acc: 0.9544 - val_loss: 0.2346 - val_acc: 0.9337
Epoch 5/10
26/26 [==============================] - 0s 9ms/step - loss: 0.1296 - acc: 0.9655 - val_loss: 0.2823 - val_acc: 0.9316
Epoch 6/10
26/26 [==============================] - 0s 9ms/step - loss: 0.1095 - acc: 0.9737 - val_loss: 0.3138 - val_acc: 0.9302
Epoch 7/10
26/26 [==============================] - 0s 9ms/step - loss: 0.0700 - acc: 0.9852 - val_loss: 0.4437 - val_acc: 0.9316
Epoch 8/10
26/26 [==============================] - 0s 9ms/step - loss: 0.0615 - acc: 0.9